# FINAL PROJECT
**Vance Russell** /
December 2023

## INTRODUCTION
I wanted to visualize and analyze age of tree stands in Canada using Geemap and the dataset [CA Forest Age](https://opendata.nfis.org/clipserver/nfis-change_eng.html) from Canada's national forest inventory system. The dataset is quite large (14gb) so viewing and analyzig via geemap is propitious. The dataset comes from Maltman et al. [2023](https://www.sciencedirect.com/science/article/pii/S0034425723000809?via%3Dihub)
## DATA SOURCES
The raster dataset comes from the Canadian NFIS and is called 'projects/sat-io/open-datasets/CA_FOREST/CA_forest_age_2019'. A number of additional datasets in this collection are quite interesting such as the leading tree species, basal area, and above ground biomass. I added the Hansen tree loss image collection to this analysis. **Fire datasets??**
## ANALYSIS
The Hansen dataset helped to analyze where there was a correlation between forest loss and age. Running a linear regression of loss vs. age. I also summed descriptive statistics for forest across the western range of Canadian forests and looked at fires from 1985-2020 and CA wildfire dnbr to see if any relationships exist between age and fire. The analysis steps are annotated with comments at the beginning of each code block and then interprested in the Output section of the project


In [1]:
# Import geemap and create an interactive map
import ee
import geemap
geemap.ee_initialize()
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

The provided color (default) is invalid. Using the default black color.
'#default' is not in web format. Need 3 or 6 hex digit.
The provided color (n03) is invalid. Using the default black color.
'#n03' is not in web format. Need 3 or 6 hex digit.
The provided color (n04) is invalid. Using the default black color.
'#n04' is not in web format. Need 3 or 6 hex digit.
The provided color (n05) is invalid. Using the default black color.
'#n05' is not in web format. Need 3 or 6 hex digit.
The provided color (n06) is invalid. Using the default black color.
'#n06' is not in web format. Need 3 or 6 hex digit.
The provided color (n07) is invalid. Using the default black color.
'#n07' is not in web format. Need 3 or 6 hex digit.
The provided color (n08) is invalid. Using the default black color.
'#n08' is not in web format. Need 3 or 6 hex digit.
The provided color (n09) is invalid. Using the default black color.
'#n09' is not in web format. Need 3 or 6 hex digit.
The provided color (n10) is inva

In [3]:
# Import color map and use viridis as the vis_param for CA forest age. Change basemap
import geemap.colormaps as cm
palette = cm.palettes.viridis
vis_params = {'min': 0, 'max': 152, 'palette': palette}
Map.add_basemap('CartoDB.DarkMatter')

CartoDB.DarkMatter has been already added before.


In [4]:
# Set map center, add age dataset then add tree age variable to map
age = ee.Image('projects/sat-io/open-datasets/CA_FOREST/CA_forest_age_2019')
Map.setCenter(-97.616,55.628,4)
Map.addLayer(age, vis_params, 'CA Forest Age')
Map

Map(bottom=812.0, center=[55.628, -97.616], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [5]:
# Add colorbar in lower right corner
Map.add_colorbar(
    vis_params,
    label="Canada Forest Age 2019",
    orientation="horizontal",
    position="bottomright",
    layer_name="age",
    transparent_bg=False,
)

## Compare forest age to forest loss
To compare Canada forest age to forest loss we add the Hansen GLAD dataset to the map. We then use the .reduceRegion() function to calculate the total number of pixels that have been lost in the last 20 years and then pick some bounding boxes of smaller area to run a regression for age vs. loss.

In [6]:
# Add the Hansen global forest loss dataset
hansen = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')
treeLossVisParam = {
  'bands': ['lossyear'],
  'min': 0,
  'max': 22,
  'palette': ['yellow', 'red']
}
Map.addLayer(hansen, treeLossVisParam, 'tree loss year', False)

In [6]:
# # Add tree loss colorbar in lower left corner if needed, uncomment to add back
# Map.add_colorbar(
#     treeLossVisParam,
#     label="Percent Forest Loss 2000-22",
#     orientation="vertical",
#     position="bottomleft",
#     layer_name="hansen",
#     transparent_bg=False,
# )
# Map

In [7]:
# Plot age vs tree loss year
import matplotlib.pyplot as plt

# convert the ee.Image object to numpy array
age_arr = age.toArray().getInfo()
hansen_arr = hansen.select('lossyear').toArray().getInfo()

In [ ]:
# plot the arrays
plt.scatter(age_arr, hansen_arr)
plt.xlabel('Age')
plt.ylabel('Tree Loss Year')
plt.title('Age vs Tree Loss Year')
plt.show()

In [9]:
# Define the aoi coordinates
polygon_coords = [[-121.35042971330375,50.877571526454375],
                  [-120.11996096330375,50.877571526454375],
                  [-120.11996096330375,51.4699284878807],
                  [-121.35042971330375,51.4699284878807],
                  [-121.35042971330375,50.877571526454375]]

# Create an Earth Engine polygon object as the aoi
aoi = ee.Geometry.Polygon(polygon_coords)

# Add the aoi to the map
style = {'color': 'white', 'width': 2, 'lineType': 'solid', 'fillColor': 'red'}
Map.addLayer(aoi, style, 'AOI')
Map.setCenter(-120.735, 51.085, 8)
Map

Map(bottom=1583.0, center=[51.085, -120.735], controls=(WidgetControl(options=['position', 'transparent_bg'], …